In [1]:
import sys
sys.path.append('..')

from src.tool.tool_registry import ToolRegistry
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent

model = ChatOpenAI(
    model="qwen3-8B", 
    base_url="http://127.0.0.1:1234/v1", 
    api_key=""
)

tool_registry = ToolRegistry()
tools = tool_registry.get_tools()

agent_executor = create_agent(model, tools)

In [2]:
chat_history = [
    SystemMessage(
    """
        You are an expert data analyst with enhanced RAG capabilities. 
        
        ENHANCED WORKFLOW:
        1. **Entity Extraction**: First extract medical entities from user queries

        AVAILABLE TOOLS:
        1. entity_extraction: Extracts medical entities and relationships from user input

        ENHANCED WORKFLOW STEPS:
        Use the tool then return the value to proceed to the next step.
    """,
    ),
    HumanMessage("Detect what language is the question input and respond in the same language."),
    AIMessage("Understood. I will detect the input language and respond accordingly.")
]

In [3]:
from langchain_core.callbacks import BaseCallbackHandler

class ToolExecutionPrinter(BaseCallbackHandler):
    def on_tool_start(self, serialized, input_str, **kwargs):
        """Run when a tool starts running."""
        tool_name = serialized.get("name")
        print(f"\n[LOG] 🛠️  Agent is entering tool: {tool_name}")
        print(f"[LOG]    Input args: {input_str}\n")

    def on_tool_end(self, output, **kwargs):
        """Run when a tool ends running."""
        print(f"[LOG] ✅ Tool execution finished.\n")

In [4]:
user_input = """
berapa banyak claim untuk heart attack
"""

final_messages = chat_history + [HumanMessage(content=user_input)]

response = agent_executor.invoke({
    "messages": final_messages,
},
config={
        "callbacks": [ToolExecutionPrinter()]
    }
)

print(f"Response: {response['messages'][-1].content}")


[LOG] 🛠️  Agent is entering tool: entity_extraction
[LOG]    Input args: {'user_query': 'berapa banyak claim untuk heart attack'}

LLM Raw Response: <think>
Okay, let's tackle this query. The user is asking "berapa banyak claim untuk heart attack" which translates to "how many claims for heart attack" in English. 

First, I need to extract entities. The main medical condition here is "heart attack," which is synonymous with "myocardial infarction." So I'll include both terms in the diagnoses list.

Next, the query mentions "claim," which in the context of insurance would relate to the number of claims made. But since there's no specific entity like an insurance company mentioned, maybe "claims" isn't a direct entity here. Wait, the user might be asking about the count of claims related to heart attacks. However, the entities section should focus on medical terms, hospitals, doctors, etc. Since "claim" here refers to the insurance claim, but the user didn't mention any specific organiz